In [7]:
import os
import numpy as np
import polars as pl
from PIL import Image
import trimesh
import open3d as o3d
from generate_data import DataGenerator

In [3]:
# Create output directory if it doesn't exist
os.makedirs("output", exist_ok=True)

# Initialize data generator
generator = DataGenerator(None, None)
demo_data = generator.generate_demonstration(demo_id=0, interpolation_method='linear')

/Users/vineethyeevani/Documents/pseudo-demo/generate_data.py:169: RuntimeWarning: divide by zero encountered in matmul
  points_world = (camera_pose @ points.T).T
/Users/vineethyeevani/Documents/pseudo-demo/generate_data.py:169: RuntimeWarning: overflow encountered in matmul
  points_world = (camera_pose @ points.T).T
/Users/vineethyeevani/Documents/pseudo-demo/generate_data.py:169: RuntimeWarning: invalid value encountered in matmul
  points_world = (camera_pose @ points.T).T


In [ ]:
# Create directories for images and pointclouds
image_dir = os.path.join("output", "images")
pointcloud_dir = os.path.join("output", "pointclouds")
os.makedirs(image_dir, exist_ok=True)
os.makedirs(pointcloud_dir, exist_ok=True)

# Save images and point clouds to files
for i, obs in enumerate(demo_data['observations']):
    for camera_id, camera_data in obs.items():
        color_path = os.path.join(image_dir, f'{camera_id}_color_{i:06d}.png')
        point_cloud_path = os.path.join(pointcloud_dir, f'{camera_id}_point_cloud_{i:06d}.pcd')
        
        # Save color and depth images as PNG using PIL
        Image.fromarray(camera_data['color']).save(color_path)
        
        # Save point cloud as PCD using open3d
        points = camera_data['point_cloud']
        point_cloud = o3d.geometry.PointCloud()
        point_cloud.points = o3d.utility.Vector3dVector(points)
        o3d.io.write_point_cloud(point_cloud_path, point_cloud)


In [30]:
import pandas as pd

# Create a list to store the data
data = []

# Iterate through the demo_data to extract paths and other fields
for i, obs in enumerate(demo_data['observations']):
    # Initialize a dictionary to store paths for each camera
    camera_paths = {}
    
    for camera_id, camera_data in obs.items():
        color_path = os.path.join(image_dir, f'{camera_id}_color_{i:06d}.png')
        point_cloud_path = os.path.join(pointcloud_dir, f'{camera_id}_point_cloud_{i:06d}.pcd')
        
        # Store paths in the dictionary with keys indicating the camera
        camera_paths[f'{camera_id}_color_path'] = color_path
        camera_paths[f'{camera_id}_point_cloud_path'] = point_cloud_path
    
    # Append the paths and fields from generate_demonstration to the data list
    data.append({
        'demo_id': demo_data['demo_id'],
        'pose': demo_data['trajectory']['poses'][i].flatten(),
        **camera_paths,  # Unpack the camera paths into the dictionary
    })

# Create a DataFrame from the data list
df = pd.DataFrame(data)
df.head()

,demo_id,pose,camera_0_color_path,camera_0_point_cloud_path,camera_1_color_path,camera_1_point_cloud_path,camera_2_color_path,camera_2_point_cloud_path,camera_3_color_path,camera_3_point_cloud_path
0,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",output/images/camera_0_color_000000.png,output/pointclouds/camera_0_point_cloud_000000...,output/images/camera_1_color_000000.png,output/pointclouds/camera_1_point_cloud_000000...,output/images/camera_2_color_000000.png,output/pointclouds/camera_2_point_cloud_000000...,output/images/camera_3_color_000000.png,output/pointclouds/camera_3_point_cloud_000000...
1,0,"[0.9998633852986374, -0.0006338921585704102, -...",output/images/camera_0_color_000001.png,output/pointclouds/camera_0_point_cloud_000001...,output/images/camera_1_color_000001.png,output/pointclouds/camera_1_point_cloud_000001...,output/images/camera_2_color_000001.png,output/pointclouds/camera_2_point_cloud_000001...,output/images/camera_3_color_000001.png,output/pointclouds/camera_3_point_cloud_000001...
2,0,"[0.9994535968951457, -0.001459119028691498, -0...",output/images/camera_0_color_000002.png,output/pointclouds/camera_0_point_cloud_000002...,output/images/camera_1_color_000002.png,output/pointclouds/camera_1_point_cloud_000002...,output/images/camera_2_color_000002.png,output/pointclouds/camera_2_point_cloud_000002...,output/images/camera_3_color_000002.png,output/pointclouds/camera_3_point_cloud_000002...
3,0,"[0.9987708018686019, -0.002475344148562729, -0...",output/images/camera_0_color_000003.png,output/pointclouds/camera_0_point_cloud_000003...,output/images/camera_1_color_000003.png,output/pointclouds/camera_1_point_cloud_000003...,output/images/camera_2_color_000003.png,output/pointclouds/camera_2_point_cloud_000003...,output/images/camera_3_color_000003.png,output/pointclouds/camera_3_point_cloud_000003...
4,0,"[0.9978152786084437, -0.0036821531825075027, -...",output/images/camera_0_color_000004.png,output/pointclouds/camera_0_point_cloud_000004...,output/images/camera_1_color_000004.png,output/pointclouds/camera_1_point_cloud_000004...,output/images/camera_2_color_000004.png,output/pointclouds/camera_2_point_cloud_000004...,output/images/camera_3_color_000004.png,output/pointclouds/camera_3_point_cloud_000004...


In [31]:
# Write the DataFrame to a Parquet file in the output directory
output_parquet_path = os.path.join("output", "demo_data.parquet")
df.to_parquet(output_parquet_path, index=False)